In [ ]:
from keras.models import model_from_json
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
import numpy as np
from pprint import pprint
from sklearn.metrics import confusion_matrix


In [ ]:
test_dir = "./dataset/val_images"
inception_json = "./model/inception_model.json"
checkpoint_path_2 = './model/checkpoint_2.h5'


In [ ]:
def main():
    test_data_gen = ImageDataGenerator(rescale=1. / 255)
    test_generator = test_data_gen.flow_from_directory(
        test_dir,
        target_size=(299, 299),
        batch_size=1,
        class_mode="categorical"
    )
    file_names = test_generator.filenames
    nb_samples = len(file_names)
    print("nb_samples %.2f%%" % nb_samples)

    json_file = open(inception_json, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights(checkpoint_path_2)

    # evaluate loaded model on test data
    sgd = SGD(lr=0.0001, momentum=0.9, nesterov=True)
    loaded_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    score = loaded_model.predict_generator(test_generator, steps=nb_samples)

    for i in range(0, nb_samples):
        idx = np.argmax(score[i])
        pprint("prediction idx, value %.2f%% %.2f%%" % (idx, score[i][idx]))

    Y_pred = loaded_model.predict_generator(test_generator, 300)
    y_pred = np.argmax(Y_pred, axis=1)
    print("confusion matrix")
    print(confusion_matrix(test_generator.classes, y_pred))


In [ ]:
if __name__ == '__main__':
    main()
